In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <08E12B12-6183-307E-BDA0-374FA8EBA2C9> /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <709C1DF5-D253-3C66-87E2-C99FD3A259DF> /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/ut

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x

class UNetPlusPlus(nn.Module):
    def __init__(self, in_channels=3, num_classes=1):
        super(UNetPlusPlus, self).__init__()
        
        # Encoder (downsampling)
        self.encoder1 = ConvBlock(in_channels, 64)
        self.encoder2 = ConvBlock(64, 128)
        self.encoder3 = ConvBlock(128, 256)
        self.encoder4 = ConvBlock(256, 512)

        # Decoder (upsampling)
        self.decoder4_0 = ConvBlock(512, 256)
        self.decoder3_0 = ConvBlock(256 + 256, 128)
        self.decoder2_0 = ConvBlock(128 + 128, 64)

        # Nested Decoder Blocks
        self.decoder3_1 = ConvBlock(256 + 128 + 128, 128)
        self.decoder2_1 = ConvBlock(128 + 64 + 64, 64)
        self.decoder2_2 = ConvBlock(64 + 64 + 64, 64)

        # Final layer
        self.final = nn.Conv2d(64, num_classes, kernel_size=1)
        
        # Pooling
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        # Encoding path
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        enc4 = self.encoder4(self.pool(enc3))
        
        # Decoding path
        dec4_0 = F.interpolate(enc4, size=enc3.shape[2:], mode='bilinear', align_corners=False)
        dec4_0 = self.decoder4_0(dec4_0)
        
        dec3_0 = torch.cat([dec4_0, enc3], dim=1)
        dec3_0 = self.decoder3_0(dec3_0)
        
        # 调整 dec4_0 和 enc2 的尺寸，使其与 dec3_0 一致
        dec3_1 = F.interpolate(dec3_0, size=enc2.shape[2:], mode='bilinear', align_corners=False)
        enc2_resized = F.interpolate(enc2, size=enc2.shape[2:], mode='bilinear', align_corners=False)
        dec4_0_resized = F.interpolate(dec4_0, size=enc2.shape[2:], mode='bilinear', align_corners=False)
        
        # 拼接尺寸调整后的张量
        dec3_1 = torch.cat([dec3_1, enc2_resized, dec4_0_resized], dim=1)
        dec3_1 = self.decoder3_1(dec3_1)

        dec2_0 = torch.cat([F.interpolate(dec3_0, size=enc2.shape[2:], mode='bilinear', align_corners=False), enc2], dim=1)
        dec2_0 = self.decoder2_0(dec2_0)
        
        dec2_1 = F.interpolate(dec3_1, size=enc1.shape[2:], mode='bilinear', align_corners=False)
        dec2_1 = torch.cat([dec2_1, enc1, F.interpolate(dec2_0, size=enc1.shape[2:], mode='bilinear', align_corners=False)], dim=1)
        dec2_1 = self.decoder2_1(dec2_1)
        
        dec2_2 = torch.cat([F.interpolate(dec2_1, size=enc1.shape[2:], mode='bilinear', align_corners=False), enc1, dec2_1], dim=1)
        dec2_2 = self.decoder2_2(dec2_2)

        # Final layer
        output = self.final(dec2_2)
        return output

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

# 自定义数据集类
class ContrailDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        super().__init__()
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.files = [f for f in self.image_dir.iterdir() if f.is_file() and f.suffix in ['.png', '.jpg', '.jpeg']]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image_path = self.files[idx]
        mask_path = self.mask_dir / image_path.name

        image = Image.open(image_path).convert('RGB')  # 读取为RGB图像
        mask = Image.open(mask_path).convert('L')  # 读取为灰度图像

        if self.transform:
            image = self.transform(image)
            mask = transforms.ToTensor()(mask)

        mask = (mask > 0).float()  # 将掩码二值化
        return image, mask.unsqueeze(0)  # 在channel维度增加一个维度

# 定义Dice损失函数
class DiceLoss(nn.Module):
    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
        return 1 - dice

# 计算 Precision, Recall, F1 Score, IoU
def calculate_metrics(preds, labels):
    preds = (torch.sigmoid(preds) > 0.5).cpu().numpy().flatten()  # 二值化预测
    labels = labels.cpu().numpy().flatten()

    precision = precision_score(labels, preds, zero_division=1)
    recall = recall_score(labels, preds, zero_division=1)
    f1 = f1_score(labels, preds, zero_division=1)
    iou = jaccard_score(labels, preds, zero_division=1)

    return precision, recall, f1, iou

# 计算平均 IoU（mIoU），对于二分类问题，mIoU 就是 IoU 本身
def calculate_mIoU(preds, labels):
    preds = (torch.sigmoid(preds) > 0.5).cpu().numpy()
    labels = labels.cpu().numpy()

    ious = []
    for i in range(preds.shape[0]):  # 针对每一张图片
        iou = jaccard_score(labels[i].flatten(), preds[i].flatten(), zero_division=1)
        ious.append(iou)
    
    return sum(ious) / len(ious)

def calculate_accuracy(preds, labels):
    # 使用Sigmoid并将输出二值化
    preds = (torch.sigmoid(preds) > 0.5).float()  # 二值化预测
    # 确保预测和标签维度一致
    preds = preds.view(-1)
    labels = labels.view(-1)

    # 计算正确的预测像素总数
    correct = (preds == labels).sum().item()
    # 计算总像素数
    total = labels.size(0)

    # 返回准确率
    acc = correct / total
    return acc

# 训练函数
def train_model(model, train_loader, valid_loader, loss_fn, optimizer, num_epochs, device):
    best_f1 = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_acc = 0

        # Training phase
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = calculate_accuracy(outputs, masks)
            train_acc += acc

        # Validation phase
        model.eval()
        valid_loss = 0
        valid_acc = 0
        valid_precision = 0
        valid_recall = 0
        valid_f1 = 0
        valid_iou = 0
        valid_miou = 0

        with torch.no_grad():
            for images, masks in valid_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)

                loss = loss_fn(outputs, masks)
                valid_loss += loss.item()

                acc = calculate_accuracy(outputs, masks)
                valid_acc += acc

                precision, recall, f1, iou = calculate_metrics(outputs, masks)
                valid_precision += precision
                valid_recall += recall
                valid_f1 += f1
                valid_iou += iou
                valid_miou += calculate_mIoU(outputs, masks)

        # 平均值
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_acc / len(train_loader)
        avg_valid_loss = valid_loss / len(valid_loader)
        avg_valid_acc = valid_acc / len(valid_loader)
        avg_valid_precision = valid_precision / len(valid_loader)
        avg_valid_recall = valid_recall / len(valid_loader)
        avg_valid_f1 = valid_f1 / len(valid_loader)
        avg_valid_iou = valid_iou / len(valid_loader)
        avg_valid_miou = valid_miou / len(valid_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}, '
              f'Val Loss: {avg_valid_loss:.4f}, Val Acc: {avg_valid_acc:.4f}, '
              f'Precision: {avg_valid_precision:.4f}, Recall: {avg_valid_recall:.4f}, '
              f'F1 Score: {avg_valid_f1:.4f}, IoU: {avg_valid_iou:.4f}, mIoU: {avg_valid_miou:.4f}')

        # 保存最好的模型
        if avg_valid_f1 > best_f1:
            best_f1 = avg_valid_f1
            torch.save(model.state_dict(), 'UNet++_model.pth')

# 设置参数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_dir = '/Users/camus/Desktop/trainX/image'
mask_dir = '/Users/camus/Desktop/trainX/maskimage'

batch_size = 4
num_epochs = 100

# 图像预处理，直接使用256x256的尺寸
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 将图像调整为256x256
    transforms.ToTensor(),
])

# 加载数据集
dataset = ContrailDataset(image_dir, mask_dir, transform=transform)
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

# 指定权重文件的路径
backbone_weights_path = '/Volumes/Vettel/学习/预训练模型/resnet50-0676ba61.pth'

# 初始化UNet++模型和损失函数
model = UNetPlusPlus(in_channels=3, num_classes=1).to(device)
loss_fn = DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
train_model(model, train_loader, valid_loader, loss_fn, optimizer, num_epochs, device)

Epoch [1/100], Train Loss: 0.5891, Train Acc: 0.8933, Val Loss: 0.8371, Val Acc: 0.9083, Precision: 1.0000, Recall: 0.0054, F1 Score: 0.0104, IoU: 0.0054, mIoU: 0.0045
Epoch [2/100], Train Loss: 0.4717, Train Acc: 0.9308, Val Loss: 0.4104, Val Acc: 0.9465, Precision: 0.6983, Recall: 0.8294, F1 Score: 0.7479, IoU: 0.6075, mIoU: 0.6354
Epoch [3/100], Train Loss: 0.3512, Train Acc: 0.9423, Val Loss: 0.5629, Val Acc: 0.7728, Precision: 0.3081, Recall: 0.9963, F1 Score: 0.4612, IoU: 0.3076, mIoU: 0.3442
Epoch [4/100], Train Loss: 0.3364, Train Acc: 0.9363, Val Loss: 0.3931, Val Acc: 0.8878, Precision: 0.4759, Recall: 0.9795, F1 Score: 0.6295, IoU: 0.4703, mIoU: 0.4933
Epoch [5/100], Train Loss: 0.2878, Train Acc: 0.9447, Val Loss: 0.3193, Val Acc: 0.9553, Precision: 0.8805, Recall: 0.6021, F1 Score: 0.7064, IoU: 0.5552, mIoU: 0.5585
Epoch [6/100], Train Loss: 0.2658, Train Acc: 0.9486, Val Loss: 0.3981, Val Acc: 0.8788, Precision: 0.4670, Recall: 0.9575, F1 Score: 0.6131, IoU: 0.4560, mIoU:

Epoch [50/100], Train Loss: 0.2071, Train Acc: 0.9548, Val Loss: 0.3506, Val Acc: 0.8963, Precision: 0.5106, Recall: 0.9565, F1 Score: 0.6497, IoU: 0.4961, mIoU: 0.5594
Epoch [51/100], Train Loss: 0.2238, Train Acc: 0.9531, Val Loss: 0.3099, Val Acc: 0.9539, Precision: 0.8774, Recall: 0.5889, F1 Score: 0.6904, IoU: 0.5415, mIoU: 0.5234
Epoch [52/100], Train Loss: 0.2277, Train Acc: 0.9532, Val Loss: 0.2443, Val Acc: 0.9526, Precision: 0.7252, Recall: 0.8035, F1 Score: 0.7560, IoU: 0.6113, mIoU: 0.6330
Epoch [53/100], Train Loss: 0.2236, Train Acc: 0.9532, Val Loss: 0.2004, Val Acc: 0.9629, Precision: 0.8036, Recall: 0.8052, F1 Score: 0.7999, IoU: 0.6720, mIoU: 0.6754
Epoch [54/100], Train Loss: 0.2203, Train Acc: 0.9554, Val Loss: 0.4046, Val Acc: 0.9448, Precision: 0.8985, Recall: 0.4633, F1 Score: 0.5957, IoU: 0.4383, mIoU: 0.4551
Epoch [55/100], Train Loss: 0.2108, Train Acc: 0.9560, Val Loss: 0.1984, Val Acc: 0.9610, Precision: 0.7597, Recall: 0.8590, F1 Score: 0.8020, IoU: 0.6750,

Epoch [99/100], Train Loss: 0.1522, Train Acc: 0.9693, Val Loss: 0.2027, Val Acc: 0.9584, Precision: 0.7437, Recall: 0.8736, F1 Score: 0.7976, IoU: 0.6703, mIoU: 0.6871
Epoch [100/100], Train Loss: 0.1473, Train Acc: 0.9699, Val Loss: 0.2090, Val Acc: 0.9565, Precision: 0.7212, Recall: 0.8929, F1 Score: 0.7911, IoU: 0.6624, mIoU: 0.6744
